In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("asag_dataset.csv")  

# Display first few rows
# print(df.head())


   \                                           question  \
0  0   Give a definition for the term "artificial ne...   
1  1   Give a definition for the term "artificial ne...   
2  2   Give a definition for the term "artificial ne...   
3  3   Give a definition for the term "artificial ne...   
4  4   Give a definition for the term "artificial ne...   

                                      student_answer  grades_round  \
0  An artificial neural network is a massively pa...             2   
1  Artificial neural network consists of: . Large...             2   
2  An artificial neural network is a massive dist...             1   
3  An ANN is a layered graphical model containing...             2   
4  Artificial Neural Networks are large parallel ...             2   

                                    student_modified  \
0  artificial neural network massively parallel d...   
1  artificial neural network consists largely par...   
2  artificial neural network massive distributed ...   


In [2]:
from sentence_transformers import SentenceTransformer

# Load SBERT model
sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2')


c:\Users\shawn\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shawn\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shawn\.cache\huggingface\hub\models--sentence-transformers--paraphrase-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode 

In [4]:
# # Function to compute SBERT embeddings
# def compute_embedding(text):
#     return model.encode(text) if isinstance(text, str) else np.zeros(384)  # 384 is SBERT's embedding size

# # Apply the function to the relevant columns
# df['embed_ref'] = df['ref_answer'].apply(compute_embedding)
# df['embed_stud'] = df['student_answer'].apply(compute_embedding)
# df['embed_ref_modified'] = df['ref_modified'].apply(compute_embedding)
# df['embed_stud_modified'] = df['student_modified'].apply(compute_embedding)
# df['embed_ref_demoted'] = df['ref_demoted'].apply(compute_embedding)
# df['embed_stud_demoted'] = df['student_demoted'].apply(compute_embedding)


In [3]:
print(df.columns)


Index(['\', 'question', 'student_answer', 'grades_round', 'student_modified',
       'ref_answer', 'qn_modified', 'ref_modified', 'student_demoted',
       'ref_demoted', 'length_ratio', 'embed_ref', 'embed_stud',
       'embed_ref_demoted', 'embed_stud_demoted', 'aligned', 'aligned_demoted',
       'cos_similarity', 'cos_similarity_demo', 'aligned_score',
       'aligned_score_demo', 'question_id'],
      dtype='object')


In [5]:
expected_columns = [
    "question", "student_answer", "ref_answer", "qn_modified", "student_modified",
    "ref_modified", "student_demoted", "ref_demoted", "length_ratio",
    "embed_ref", "embed_stud", "embed_ref_modified", "embed_stud_modified",
    "embed_ref_demoted", "embed_stud_demoted", "aligned", "aligned_demoted",
    "grades_round", "cos_similarity", "cos_similarity_modified", "cos_similarity_demo",
    "aligned_score", "aligned_score_demo", "question_id"
]

missing_columns = [col for col in expected_columns if col not in df.columns]

print("Missing Columns:", missing_columns)


Missing Columns: ['embed_ref_modified', 'embed_stud_modified', 'cos_similarity_modified']


In [7]:
# Ensure all values are strings and handle NaNs
df["embed_ref_modified"] = df["ref_modified"].astype(str).apply(lambda x: sbert_model.encode(x).tolist() if x.strip() else None)
df["embed_stud_modified"] = df["student_modified"].astype(str).apply(lambda x: sbert_model.encode(x).tolist() if x.strip() else None)


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Convert embeddings back to NumPy arrays and compute cosine similarity
df['cos_similarity_modified'] = df.apply(
    lambda row: cosine_similarity([row['embed_ref_modified']], [row['embed_stud_modified']])[0][0],
    axis=1
)


In [ ]:
# #CHECK AGAIN
# expected_columns = [
#     "question", "student_answer", "ref_answer", "qn_modified", "student_modified",
#     "ref_modified", "student_demoted", "ref_demoted", "length_ratio",
#     "embed_ref", "embed_stud", "embed_ref_modified", "embed_stud_modified",
#     "embed_ref_demoted", "embed_stud_demoted", "aligned", "aligned_demoted",
#     "grades_round", "cos_similarity", "cos_similarity_modified", "cos_similarity_demo",
#     "aligned_score", "aligned_score_demo", "question_id"
# ]

# missing_columns = [col for col in expected_columns if col not in df.columns]

# print("Missing Columns:", missing_columns)


Missing Columns: []


In [12]:
print(df.head())


   \                                           question  \
0  0   Give a definition for the term "artificial ne...   
1  1   Give a definition for the term "artificial ne...   
2  2   Give a definition for the term "artificial ne...   
3  3   Give a definition for the term "artificial ne...   
4  4   Give a definition for the term "artificial ne...   

                                      student_answer  grades_round  \
0  An artificial neural network is a massively pa...             2   
1  Artificial neural network consists of: . Large...             2   
2  An artificial neural network is a massive dist...             1   
3  An ANN is a layered graphical model containing...             2   
4  Artificial Neural Networks are large parallel ...             2   

                                    student_modified  \
0  artificial neural network massively parallel d...   
1  artificial neural network consists largely par...   
2  artificial neural network massive distributed ...   


In [ ]:

# # Assuming your DataFrame is called 'df'
# df.to_csv('new_file.csv', index=False)

In [18]:
def assign_grades(cos_sim):
    if cos_sim > 0.69:
        return "Completely Correct", 2
    elif cos_sim > 0.395998348:
        return "Partially Incorrect", 1
    else:
        return "Incorrect", 0

df[["grade_text", "grades_auto"]] = df["cos_similarity_modified"].apply(lambda x: pd.Series(assign_grades(x)))


In [ ]:
# df.to_csv('new_file_new.csv', index=False)

In [ ]:
from bert_score import score

# Compute BERTScore
P, R, F1 = score(df["student_modified"].tolist(), df["ref_modified"].tolist(), lang="en", model_type="roberta-large")

df["bert_score"] = F1.numpy()


c:\Users\shawn\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shawn\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\shawn\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_dow

AttributeError: 'float' object has no attribute 'split'